In [ ]:
# [예제 4.27] 지역고용통계 데이터 읽어오기

import pandas as pd

# job_path = r'C:\Users\JK\Desktop\부동산 데이터\지역고용통계.xls'
job_path = r' 고용 데이터 파일 디렉터리 \ 지역고용통계.xls'
job_raw = pd.read_excel(job_path, skiprows=2)

In [ ]:
# [예제 4.28] 데이터프레임 다듬기

job_data = job_raw[job_raw['산업별'] == '전산업']
job_data = job_data[['지역별', '전체종사자 (명)']]
job_data.columns = ['지역명',  '고용자수']

In [ ]:
# [예제 4.29] 지역명 나누기(ch04/ 4.2 일자리.ipynb)

def get_sido(x):
    temp = x.split(' ')[0]
    if len(temp) != 4:
        return temp[:2]
    else:
        return temp[0] + temp[2]

job_data['시도'] = job_data['지역명'].apply(get_sido)
job_data['구군'] = job_data['지역명'].apply(lambda x: x.split(' ')[1])

In [ ]:
# [예제 4.30] 지역고용통계 전처리 과정 정리해서 함수화

def job_preprocessing(path):
    job_raw = pd.read_excel(path, skiprows=2)
    job_data = job_raw[job_raw['산업별'] == '전산업']
    job_data = job_data[['지역별', '전체종사자 (명)']]
    job_data.columns = ['지역명',  '고용자수']
    
    def get_sido(x):
        temp = x.split(' ')[0]
        if len(temp) != 4:
            return temp[:2]
        else:
            return temp[0] + temp[2]

    job_data['시도'] = job_data['지역명'].apply(get_sido)
    job_data['구군'] = job_data['지역명'].apply(lambda x: x.split(' ')[1])
    
    return job_data

In [ ]:
# [예제 4.31] 세대수 엑셀 데이터 읽어오기

# house_n_path = r'C:\Users\JK\Desktop\부동산 데이터\행정구역_시군구_별_주민등록세대수_20190107134842.xlsx'
house_n_path = r' 세대수 데이터 디렉터리 \ 세대수 데이터 엑셀파일명.xlsx'
house_n_raw = pd.read_excel(house_n_path)

In [ ]:
# [예제 4.32] 세대수 데이터프레임에서 시도명 고치기

house_n_raw.columns = ['시도', '구군', '세대수']

big_col = []
for num, temp in enumerate(house_n_raw['시도']):
    if pd.isna(temp) :
        big_col.append(big_col[num-1])
    else:
        big_col.append(temp)

house_n_raw['시도'] = big_col

In [ ]:
# [예제 4.33] 세대수 데이터프레임에서 시도명 줄임말로 고치기

def get_sido(x):
    if len(x) != 4:
        return x[:2]
    else:
        return x[0] + x[2]

house_n_raw['시도'] = house_n_raw['시도'].apply(get_sido)
house_n_data = house_n_raw[house_n_raw['구군'] != '소계']

In [ ]:
# [예제 4.34] 세대수 데이터 전처리 과정 함수화(ch04/ 4.2 일자리.ipynb)

def house_number_preprocessing(path):
    house_n_raw = pd.read_excel(path)
    house_n_raw.columns = ['시도', '구군', '세대수']

    big_col = []
    for num, temp in enumerate(house_n_raw['시도']):
        if pd.isna(temp) :
            big_col.append(big_col[num-1])
        else:
            big_col.append(temp)

    house_n_raw['시도'] = big_col
    
    def get_sido(x):
        if len(x) != 4:
            return x[:2]
        else:
            return x[0] + x[2]

    house_n_raw['시도'] = house_n_raw['시도'].apply(get_sido)
    house_n_data = house_n_raw[house_n_raw['구군'] != '소계']
    
    return house_n_data

In [ ]:
# [예제 4.35] 시도 단위로 고용자수 보기

job_path = r' 고용 데이터 파일 디렉터리 \ 지역고용통계.xls'
job_df = job_preprocessing(job_path)

job_sido = job_df.groupby('시도').sum()
job_sido = job_sido.sort_values(by='고용자수', ascending=False)

In [ ]:
# [예제 4.36] 시도 단위의 고용자수를 막대 그래프로 보기 

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
import seaborn as sns
style.use('ggplot')
%matplotlib inline

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
# 맥OS인 경우 위 두 줄을 입력하지 말고 아래 코드를 입력하세요
# rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False


plt.figure(figsize=(12, 4))
job_sido['고용자수'].plot(kind='bar', color='darkcyan')
plt.axhline(y=job_sido['고용자수'].mean(), color='orange', linewidth=2, ls='--')
plt.xticks(rotation=0)
plt.xlabel('')
plt.ylabel('고용자수')
plt.show()

In [ ]:
# [예제 4.37] 세대수 데이터 추가

house_n_path = r' 세대수 데이터 디렉터리 \ 세대수 데이터 엑셀파일명.xlsx'
house_n_df = house_number_preprocessing(house_n_path)

job_sido['세대수'] = house_n_df.groupby('시도').sum().loc[job_sido.index]
job_sido['세대수대비고용'] = job_sido['고용자수']/job_sido['세대수'] * 100

In [ ]:
# [예제 4.38] 시도 단위 세대구 대비 고용비율 막대 그래프로 보기

plt.figure(figsize=(12, 4))
job_sido.sort_values(by='세대수대비고용', ascending=False)['세대수대비고용'].plot(kind='bar', color='darkcyan')
plt.axhline(y=job_sido['세대수대비고용'].mean(), color='orange', linewidth=2, ls='--')
plt.xticks(rotation=0)
plt.xlabel('')
plt.ylabel('세대수 대비 고용자수 (%)')
plt.show()

In [ ]:
# [예제 4.39] 평균 아파트 매매가격 가져오기

def gamjungwon_price_preprocessing(path):
    row_price = pd.read_excel(path, skiprows=10)
    
    big_col = []
    for num, temp in enumerate(row_price['지 역']):
        if pd.isna(temp) :
            big_col.append(big_col[num-1])
        else:
            big_col.append(temp)


    small_col = []
    for num in range(len(row_price)):
        temp_list = list(row_price[['지 역', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']].iloc[num])
        for temp in temp_list[3::-1]:
            if not pd.isna(temp):
                small_col.append(temp)
                break

    row_price.index = [big_col, small_col]
    
    transposed_price = row_price.drop(['지 역', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'], axis=1).T
    
    time_index = []
    for time in transposed_price.index:
        temp = time.split(' ')
        time_index.append(temp[0][:-1]+'.'+temp[1][:-1])

    transposed_price.index = pd.to_datetime(time_index)
    
    return transposed_price


price_path =  r' 평균 아파트 매매가격 데이터 디렉터리 \ 평균 아파트 매매가격 데이터 엑셀 파일명.xlsx'
price_df = gamjungwon_price_preprocessing(price_path)

In [ ]:
# [예제 4.40] 고용자수, 세대수, 평균 아파트 매매가격 scatter 그래프로 나타내기

sido_list = []
for i in job_sido.index:
    sido_list.append(price_df.loc['2018-6-1'][i][i])
      
job_sido['평균매매가격'] = sido_list

plt.figure(figsize=(10, 6))
plt.scatter(job_sido['고용자수'], job_sido['세대수'], c=job_sido['평균매매가격'], s=job_sido['평균매매가격']*0.001, cmap="YlOrRd", alpha=0.5 )
plt.xlabel('고용자수')
plt.ylabel('세대수')
for name in job_sido.index:
    plt.text(job_sido['고용자수'][name]*1.01, job_sido['세대수'][name]*1.05, name, fontsize=13)

plt.colorbar()
plt.show()

In [ ]:
# [예제 4.41] 세대수 대비 고용비율, 세대수, 평균 아파트 매매가격 scatter 그래프로 나타내기

plt.figure(figsize=(10, 6))
plt.scatter(job_sido['세대수대비고용'], job_sido['세대수'], c=job_sido['평균매매가격'], s=job_sido['평균매매가격']*0.001, cmap="YlOrRd", alpha=0.5 )
plt.xlabel('세대수 대비 고용자수 (%)')
plt.ylabel('세대수')
for name in job_sido.index:
    plt.text(job_sido['세대수대비고용'][name]*1.01, job_sido['세대수'][name]*1.05, name, fontsize=13)

plt.colorbar()
plt.show()

In [ ]:
# [예제 4.42] 서울시 고용자수, 세대수, 평균 매매가격 데이터프레임 만들기

local = '서울'

job_gugun = job_df[job_df['시도']==local].groupby('구군').sum()
job_gugun['세대수'] = house_n_df[house_n_df['시도'] == local].groupby('구군').sum().loc[job_gugun.index]
job_gugun['세대수대비고용'] = job_gugun['고용자수']/job_gugun['세대수'] * 100
job_gugun['평균매매가격'] = price_df.loc['2018-6-1'][local][job_gugun.index]
job_gugun = job_gugun.dropna()

In [ ]:
# [예제 4.43] 서울시의 고용자수, 세대수 대비 고용비율 막대 그래프 그리기

plt.figure(figsize=(12, 4))
job_gugun.sort_values(by='고용자수', ascending=False)['고용자수'].plot(kind='bar', color='darkcyan')
plt.axhline(y=job_gugun['고용자수'].mean(), color='orange', linewidth=2, ls='--')
plt.xticks(rotation=45)
plt.xlabel('')
plt.ylabel('고용자수')
plt.show()

plt.figure(figsize=(12, 4))
job_gugun.sort_values(by='세대수대비고용', ascending=False)['세대수대비고용'].plot(kind='bar', color='darkcyan')
plt.axhline(y=job_sido['세대수대비고용'].mean(), color='orange', linewidth=2, ls='--')
plt.xticks(rotation=45)
plt.xlabel('')
plt.ylabel('세대수 대비 고용자수 (%)')
plt.show()

In [ ]:
# [예제 4.44] 서울시의 고용자수, 세대수, 평균 아파트 매매가로 scatter 그래프 그리기(ch04/ 4.2 일자리.ipynb)

plt.figure(figsize=(10, 6))
plt.scatter(job_gugun['고용자수'], job_gugun['세대수'], c=job_gugun['평균매매가격'], s=pd.to_numeric(job_gugun['평균매매가격'])*0.001, cmap="YlOrRd", alpha=0.5)
plt.xlabel('고용자수')
plt.ylabel('세대수')

for name in job_gugun.index:
    plt.text(job_gugun['고용자수'][name]*1.01, job_gugun['세대수'][name], name)
    
plt.colorbar()
plt.show()

In [ ]:
# [예제 4.45] 서울시의 고용비율, 세대수, 평균 아파트 매매가로 scatter 그래프 그리기

plt.figure(figsize=(10, 6))
plt.scatter(job_gugun['세대수대비고용'], job_gugun['세대수'], c=job_gugun['평균매매가격'], s=pd.to_numeric(job_gugun['평균매매가격'])*0.001, cmap="YlOrRd", alpha=0.5)
plt.xlabel('세대수 대비 고용자수 (%)')
plt.ylabel('세대수')

for name in job_gugun.index:
    plt.text(job_gugun['세대수대비고용'][name]*1.01, job_gugun['세대수'][name], name)
    
plt.colorbar()
plt.show()